In [12]:
import numpy as np
import os 
import tensorflow as tf
from tensorflow import keras as kr
import torch
import torch.utils.data as Data #将数据分批次需要用到
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init

In [13]:
#读取词汇表
def read_vocab(vocab_dir):
    with open(vocab_dir,'r',encoding = 'utf-8',errors = 'ignore') as fp:
        words = [_.strip() for _ in fp.readlines()]
    word_to_id = dict(zip(words,range(len(words)))) 
    return words, word_to_id 

words, word_to_id = read_vocab('../input/cnews.vocab.txt')
print(word_to_id)

{'<PAD>': 0, '，': 1, '的': 2, '。': 3, '一': 4, '是': 5, '在': 6, '0': 7, '有': 8, '不': 9, '了': 10, '中': 11, '1': 12, '人': 13, '大': 14, '、': 15, '国': 16, '': 3903, '2': 18, '这': 19, '上': 20, '为': 21, '个': 22, '“': 23, '”': 24, '年': 25, '学': 26, '时': 27, '我': 28, '地': 29, '和': 30, '以': 31, '到': 32, '出': 33, '来': 34, '会': 35, '行': 36, '发': 37, '：': 38, '对': 39, '们': 40, '要': 41, '生': 42, '家': 43, '他': 44, '能': 45, '也': 46, '业': 47, '金': 48, '3': 49, '成': 50, '可': 51, '分': 52, '多': 53, '现': 54, '5': 55, '就': 56, '场': 57, '新': 58, '后': 59, '于': 60, '下': 61, '日': 62, '经': 63, '市': 64, '前': 65, '过': 66, '方': 67, '得': 68, '作': 69, '月': 70, '最': 71, '开': 72, '房': 73, '》': 74, '《': 75, '高': 76, '9': 77, '8': 78, '.': 79, '而': 80, '比': 81, '公': 82, '4': 83, '说': 84, ')': 85, '将': 86, '(': 87, '都': 88, '资': 89, 'e': 90, '6': 91, '基': 92, '用': 93, '面': 94, '产': 95, '还': 96, '自': 97, '者': 98, '本': 99, '之': 100, '美': 101, '很': 102, '同': 103, '7': 105, '部': 106, '进': 107, '但': 108, '主': 109, '外': 110, '动':

In [14]:
#读取分类目录，固定
def read_category():
    categories = ['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
    categories = [x for x in categories]
    cat_to_id = dict(zip(categories,range(len(categories))))
    return categories, cat_to_id

# 获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
print(cat_to_id)

{'体育': 0, '财经': 1, '房产': 2, '家居': 3, '教育': 4, '科技': 5, '时尚': 6, '时政': 7, '游戏': 8, '娱乐': 9}


In [15]:
# 将文件转换为id表示
def process_file(filename,word_to_id,cat_to_id,max_length=600):
    contents, labels = [],[]
    with open(filename,'r',encoding = 'utf-8',errors = 'ignore') as f:
        for line in f:
            try:
                label,content = line.strip().split('\t')
                if content:
                    contents.append(list(content))
                    labels.append(label)
            except:
                pass
    data_id, label_id = [],[]
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]]) #每句话对应的类别的id
    ## 使用keras提供的pad_sequences 来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id,max_length)
    # 将标签转化为one-hot表示
    y_pad = kr.utils.to_categorical(label_id, num_classes = len(cat_to_id))
    return x_pad, y_pad

# 数据加载及分批
# 获取训练数据每个字的id和对应标签的one-hot形式
x_train, y_train = process_file('../input/cnews.train.txt', word_to_id, cat_to_id, 600)
print('x_train=', x_train)
x_val, y_val = process_file('../input/cnews.val.txt', word_to_id, cat_to_id, 600)

x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


# data batch

In [16]:
# TextRNN Model
# 文本分类，RNN模型
class TextRNN(nn.Module):   
    def __init__(self):
        super(TextRNN, self).__init__()
        # 进行词嵌入
        self.embedding = nn.Embedding(5000, 64)  
        #self.rnn = nn.LSTM(input_size=64, hidden_size=128, num_layers=2, bidirectional=True)
        self.rnn = nn.GRU(input_size=64, hidden_size=128, bidirectional=True, batch_first = True,dropout = 0.5)
        self.fc = nn.Sequential(nn.Linear(256,10),
                                #nn.Dropout(0.8),
                                nn.Softmax())
        #self.f2 = nn.Sequential(nn.Linear(128,10),
                                #nn.Softmax())
 
    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.rnn(x)
        x = F.dropout(x,p=0.8)
        x = self.fc(x[:,-1,:])
        return x

In [17]:
train_dataset = Data.TensorDataset(torch.LongTensor(x_train),torch.LongTensor(y_train))
train_loader = Data.DataLoader(dataset = train_dataset,batch_size = 100,
                              shuffle = True)
val_dataset = Data.TensorDataset(torch.LongTensor(x_val),torch.LongTensor(y_val))
val_loader = Data.DataLoader(dataset = val_dataset,batch_size = 100,
                              shuffle = True)

In [18]:
# 设置初始化方式
def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_out')
        init.constant_(m.bias.data, 0.0)

In [23]:
def train():
    #使用TextRNN
    model = TextRNN().cuda()
    model.apply(weights_init_kaiming)
    #损失函数
    Loss = nn.MultiLabelSoftMarginLoss()
    optimizer = optim.Adam(model.parameters(),lr = 0.001)
    best_val_acc = 0
    
    for epoch in range(30):
        print('epoch = ', epoch)
        for step, (x_batch,y_batch) in enumerate(train_loader):
            x = x_batch.cuda()
            y = y_batch.cuda()
            out =  model(x)
            loss = Loss(out,y)
            print('loss = ',loss)
            #反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #计算准确率
            accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
            print(accuracy)
            
            #对模型进行验证
            if (epoch+1) % 25 == 0:
                #accuracy.append(accuracy_)
                for step,(x_batch,y_batch) in enumerate(val_loader):
                    x = x_batch.cuda()
                    y = y_batch.cuda()
                    out = model(x)
                    #计算准确率
                    val_accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
                    if val_accuracy > best_val_acc:
                        torch.save(model.state_dict(),'model_params.pkl')
                        best_val_acc = val_accuracy
    print(val_accuracy)

In [24]:
# 设置随机数种子
torch.manual_seed(33)
train()

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


epoch =  0
loss =  tensor(0.7351, device='cuda:0', grad_fn=<MeanBackward0>)


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.13
loss =  tensor(0.7296, device='cuda:0', grad_fn=<MeanBackward0>)
0.19
loss =  tensor(0.7271, device='cuda:0', grad_fn=<MeanBackward0>)
0.26
loss =  tensor(0.7296, device='cuda:0', grad_fn=<MeanBackward0>)
0.18
loss =  tensor(0.7263, device='cuda:0', grad_fn=<MeanBackward0>)
0.28
loss =  tensor(0.7199, device='cuda:0', grad_fn=<MeanBackward0>)
0.34
loss =  tensor(0.7213, device='cuda:0', grad_fn=<MeanBackward0>)
0.31
loss =  tensor(0.7224, device='cuda:0', grad_fn=<MeanBackward0>)
0.28
loss =  tensor(0.7252, device='cuda:0', grad_fn=<MeanBackward0>)
0.24
loss =  tensor(0.7179, device='cuda:0', grad_fn=<MeanBackward0>)
0.32
loss =  tensor(0.7181, device='cuda:0', grad_fn=<MeanBackward0>)
0.31
loss =  tensor(0.7150, device='cuda:0', grad_fn=<MeanBackward0>)
0.33
loss =  tensor(0.7111, device='cuda:0', grad_fn=<MeanBackward0>)
0.42
loss =  tensor(0.7177, device='cuda:0', grad_fn=<MeanBackward0>)
0.35
loss =  tensor(0.7148, device='cuda:0', grad_fn=<MeanBackward0>)
0.38
loss =  tensor(

0.46
loss =  tensor(0.7073, device='cuda:0', grad_fn=<MeanBackward0>)
0.46
loss =  tensor(0.7074, device='cuda:0', grad_fn=<MeanBackward0>)
0.44
loss =  tensor(0.7012, device='cuda:0', grad_fn=<MeanBackward0>)
0.52
loss =  tensor(0.7070, device='cuda:0', grad_fn=<MeanBackward0>)
0.47
loss =  tensor(0.7068, device='cuda:0', grad_fn=<MeanBackward0>)
0.45
loss =  tensor(0.7010, device='cuda:0', grad_fn=<MeanBackward0>)
0.55
loss =  tensor(0.7097, device='cuda:0', grad_fn=<MeanBackward0>)
0.41
loss =  tensor(0.7014, device='cuda:0', grad_fn=<MeanBackward0>)
0.52
loss =  tensor(0.7006, device='cuda:0', grad_fn=<MeanBackward0>)
0.56
loss =  tensor(0.7072, device='cuda:0', grad_fn=<MeanBackward0>)
0.47
loss =  tensor(0.7026, device='cuda:0', grad_fn=<MeanBackward0>)
0.51
loss =  tensor(0.7064, device='cuda:0', grad_fn=<MeanBackward0>)
0.49
loss =  tensor(0.6981, device='cuda:0', grad_fn=<MeanBackward0>)
0.58
loss =  tensor(0.7023, device='cuda:0', grad_fn=<MeanBackward0>)
0.51
loss =  tensor(

...

loss =  tensor(0.6571, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6592, device='cuda:0', grad_fn=<MeanBackward0>)
0.96
epoch =  29
loss =  tensor(0.6552, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6593, device='cuda:0', grad_fn=<MeanBackward0>)
0.96
loss =  tensor(0.6572, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6552, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6572, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6555, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6582, device='cuda:0', grad_fn=<MeanBackward0>)
0.97
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6594, device='cuda:0', grad_fn=<MeanBackward0>)
0.96
loss =  tensor(0.6554, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6563, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tens

0.98
loss =  tensor(0.6553, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6590, device='cuda:0', grad_fn=<MeanBackward0>)
0.96
loss =  tensor(0.6563, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6574, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6573, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6565, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6570, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6563, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6568, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6566, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0

loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6552, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6581, device='cuda:0', grad_fn=<MeanBackward0>)
0.97
loss =  tensor(0.6582, device='cuda:0', grad_fn=<MeanBackward0>)
0.97
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6572, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6567, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6582, device='cuda:0', grad_fn=<MeanBackward0>)
0.97
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6567, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6571

loss =  tensor(0.6552, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6563, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6552, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6591, device='cuda:0', grad_fn=<MeanBackward0>)
0.96
loss =  tensor(0.6563, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6573, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6563, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6552, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6572, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6581, device='cuda:0', grad_fn=<MeanBackward0>)
0.97
loss =  tensor(0.6553, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6572, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6580, device='cuda:0', grad_fn=<MeanBackward0>)
0.97
loss =  tensor(0.6562, d

loss =  tensor(0.6572, device='cuda:0', grad_fn=<MeanBackward0>)
0.98
loss =  tensor(0.6552, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6582, device='cuda:0', grad_fn=<MeanBackward0>)
0.97
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6592, device='cuda:0', grad_fn=<MeanBackward0>)
0.96
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6581, device='cuda:0', grad_fn=<MeanBackward0>)
0.97
loss =  tensor(0.6552, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6590, device='cuda:0', grad_fn=<MeanBackward0>)
0.96
loss =  tensor(0.6555, device='cuda:0', grad_fn=<MeanBackward0>)
1.0
loss =  tensor(0.6562, device='cuda:0', grad_fn=<MeanBackward0>)
0.99
loss =  tensor(0.6559, 